In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-10 20:35:02--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-11-10 20:35:03 (9.91 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CombineETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [7]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url="https://storage.googleapis.com/big-data-bowl/nflcombine.csv"
spark.sparkContext.addFile(url)
combine_data_df = spark.read.csv(SparkFiles.get("nflcombine.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
combine_data_df.show()

+--------------------+---+--------------+----+---+----+--------+-----+---------+-----+-------+--------------------+-----+----+----+
|              Player|Pos|        School|  Ht| Wt|40yd|Vertical|Bench|broadJump|3Cone|Shuttle|                Team|Round|Pick|Year|
+--------------------+---+--------------+----+---+----+--------+-----+---------+-----+-------+--------------------+-----+----+----+
|      Courtney Brown| DE|    Penn State| 6-5|269|4.78|    null| null|     null| null|   null|   Cleveland Browns |    1|   1|2000|
|     LaVar Arrington|OLB|    Penn State| 6-3|250|4.53|    null| null|     null| null|   null|Washington Redskins |    1|   2|2000|
|       Chris Samuels| OT|       Alabama| 6-5|325|5.08|    null| null|     null| null|   null|Washington Redskins |    1|   3|2000|
|       Peter Warrick| WR| Florida State|5-11|194|4.58|    null| null|     null| null|   null| Cincinnati Bengals |    1|   4|2000|
|         Jamal Lewis| RB|     Tennessee| 6-0|240|4.58|    null|   23|     n

In [8]:
combine_data_df.dtypes

[('Player', 'string'),
 ('Pos', 'string'),
 ('School', 'string'),
 ('Ht', 'string'),
 ('Wt', 'int'),
 ('40yd', 'double'),
 ('Vertical', 'double'),
 ('Bench', 'int'),
 ('broadJump', 'int'),
 ('3Cone', 'double'),
 ('Shuttle', 'double'),
 ('Team', 'string'),
 ('Round', 'int'),
 ('Pick', 'int'),
 ('Year', 'int')]

Postgres Setup

In [9]:
# Store environment variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for Cloud SQL
mode = "append"
jdbc_url="jdbc:postgresql://34.72.136.99:5432/big-data-bowl"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [11]:
# Write DataFrame to games table in Cloud SQL
combine_data_df.write.jdbc(url=jdbc_url, table='combine', mode=mode, properties=config)

In [12]:
df = spark.read.jdbc(url=jdbc_url,table='combine',properties=config)

In [13]:
df.show()

+--------------------+---+--------------+----+---+----+--------+-----+---------+-----+-------+--------------------+-----+----+----+
|              player|pos|        school|  ht| wt|40yd|vertical|bench|broadjump|3cone|shuttle|                team|round|pick|year|
+--------------------+---+--------------+----+---+----+--------+-----+---------+-----+-------+--------------------+-----+----+----+
|      Courtney Brown| DE|    Penn State| 6-5|269|4.78|    null| null|     null| null|   null|   Cleveland Browns |    1|   1|2000|
|     LaVar Arrington|OLB|    Penn State| 6-3|250|4.53|    null| null|     null| null|   null|Washington Redskins |    1|   2|2000|
|       Chris Samuels| OT|       Alabama| 6-5|325|5.08|    null| null|     null| null|   null|Washington Redskins |    1|   3|2000|
|       Peter Warrick| WR| Florida State|5-11|194|4.58|    null| null|     null| null|   null| Cincinnati Bengals |    1|   4|2000|
|         Jamal Lewis| RB|     Tennessee| 6-0|240|4.58|    null|   23|     n